## Part 1 - Produce Data
We elected to use the datagen connector to generate fake data for this assignment. The topic we used was 'stocktrades. The steps were as follows:
*  Open a browser and go to http://localhost:9021/
*  Select the available cluster
*  On the menu bar, select Connect
*  Click on the connect-default cluster in the Connect Clusters list.
*  Click on Add connector
*  Select DatagenConnector
*  Enter connector_stock_trades in the Name field

Then:
Generate a data stream with following configurations:
```
Key converter class: org.apache.kafka.connect.storage.StringConverter
kafka.topic: stocktrades
max.interval: 100
quickstart: Stock_Trades
```


## Part 2 - Using Ksql to create at least 2 streams with filtering from topics

To begin, you need to create a stream called stocktrades with no filtering in place.

In [ ]:
CREATE STREAM STOCKTRADES
   (SIDE STRING, QUANTITY INTEGER, SYMBOL STRING, PRICE INTEGER, ACCOUNT STRING, USERID STRING)
       WITH (KAFKA_TOPIC='stocktrades', VALUE_FORMAT='AVRO');

### Stream 1 - Sell Stream
It may be in the interest of the business to view only streams where the stock was sold and not bought. This would be useful in identifying which shares should be taken as a 'short' position

In [ ]:
CREATE STREAM SELL_TRADES WITH (KAFKA_TOPIC='SELL_TRADES', PARTITIONS=1, REPLICAS=1) AS SELECT
  STOCKTRADES.QUANTITY QUANTITY,
  STOCKTRADES.SYMBOL SYMBOL,
  STOCKTRADES.PRICE PRICE,
  STOCKTRADES.ACCOUNT ACCOUNT,
  STOCKTRADES.USERID USERID
FROM STOCKTRADES STOCKTRADES
WHERE (STOCKTRADES.SIDE = 'SELL')
EMIT CHANGES;


### Stream 2 - Buy Stream
It may also be interesting to the business to see trades that were large buys.

In [ ]:
CREATE STREAM BUY_TRADES WITH (KAFKA_TOPIC='BUY_TRADES', PARTITIONS=1, REPLICAS=1) AS SELECT
  STOCKTRADES.QUANTITY QUANTITY,
  STOCKTRADES.SYMBOL SYMBOL,
  STOCKTRADES.PRICE PRICE,
  STOCKTRADES.ACCOUNT ACCOUNT,
  STOCKTRADES.USERID USERID
FROM STOCKTRADES STOCKTRADES
WHERE (STOCKTRADES.SIDE = 'BUY')
EMIT CHANGES;

### Table 1 - Aggregated Buy Trades


In [ ]:
CREATE TABLE AGG_BUY_ORDERS WITH (KAFKA_TOPIC='AGG_BUY_ORDERS', PARTITIONS=1, REPLICAS=1) AS SELECT
  BUY_TRADES.SYMBOL SYMBOL,
  SUM(BUY_TRADES.QUANTITY) QUANTITY_AGG,
  AVG(BUY_TRADES.PRICE) PRICE_AVG,
  SUM((BUY_TRADES.QUANTITY * BUY_TRADES.PRICE)) VALUE_TRADED
FROM BUY_TRADES BUY_TRADES
WINDOW TUMBLING ( SIZE 60 SECONDS )
GROUP BY BUY_TRADES.SYMBOL
EMIT CHANGES;


### Table 2 - Aggregated Sell Trades

In [ ]:
CREATE TABLE AGG_SELL_ORDERS WITH (KAFKA_TOPIC='AGG_SELL_ORDERS', PARTITIONS=1, REPLICAS=1) AS SELECT
  SELL_TRADES.SYMBOL SYMBOL,
  SUM(SELL_TRADES.QUANTITY) QUANTITY_AGG,
  AVG(SELL_TRADES.PRICE) PRICE_AVG,
  SUM((SELL_TRADES.QUANTITY * SELL_TRADES.PRICE)) VALUE_TRADED
FROM SELL_TRADES SELL_TRADES
WINDOW TUMBLING ( SIZE 60 SECONDS )
GROUP BY SELL_TRADES.SYMBOL
EMIT CHANGES;

## Part 3 - Consume/Transform data with Spark Streaming

In [143]:
from pyspark.sql import SparkSession
from IPython.display import display, clear_output
import time
from pyspark.sql import functions as F
from pyspark.sql.types import StructType,StringType, StructField, IntegerType, FloatType

In [144]:
spark = SparkSession.builder \
        .appName('kafka') \
        .getOrCreate()

In [145]:
spark.version

'3.1.1'

In [146]:
spark._jvm.org.apache.hadoop.util.VersionInfo.getVersion()

'3.2.0'

## Raw Data Streams

In [147]:
stream_df = spark \
  .readStream \
  .format("kafka") \
  .option("kafka.bootstrap.servers", "broker:29092") \
  .option("startingOffsets", "earliest") \
  .option("subscribe", "stocktrades") \
  .load()

In [148]:
stream_df.printSchema()

root
 |-- key: binary (nullable = true)
 |-- value: binary (nullable = true)
 |-- topic: string (nullable = true)
 |-- partition: integer (nullable = true)
 |-- offset: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- timestampType: integer (nullable = true)



In [149]:
raw_stream = stream_df \
    .writeStream \
    .format("memory") \
    .queryName("raw_stocktrades_view") \
    .start()

In [151]:
clear_output(wait=True)
display(spark.sql('SELECT key, VALUE FROM raw_stocktrades_view').show(20,False))
time.sleep(1)

+----------------+-------------------------------------------------------------------------------------------------------+
|key             |VALUE                                                                                                  |
+----------------+-------------------------------------------------------------------------------------------------------+
|[5A 56 5A 5A 54]|[00 00 00 00 01 06 42 55 59 90 0B 0A 5A 56 5A 5A 54 BE 0E 0C 41 42 43 31 32 33 0C 55 73 65 72 5F 31]   |
|[5A 56 5A 5A 54]|[00 00 00 00 01 06 42 55 59 96 38 0A 5A 56 5A 5A 54 92 01 0C 4C 4D 4E 34 35 36 0C 55 73 65 72 5F 38]   |
|[5A 57 5A 5A 54]|[00 00 00 00 01 08 53 45 4C 4C AE 29 0A 5A 57 5A 5A 54 D6 06 0C 41 42 43 31 32 33 0C 55 73 65 72 5F 32]|
|[5A 4A 5A 5A 54]|[00 00 00 00 01 06 42 55 59 B8 29 0A 5A 4A 5A 5A 54 AC 06 0C 58 59 5A 37 38 39 0C 55 73 65 72 5F 38]   |
|[5A 56 5A 5A 54]|[00 00 00 00 01 06 42 55 59 D0 43 0A 5A 56 5A 5A 54 84 0F 0C 58 59 5A 37 38 39 0C 55 73 65 72 5F 34]   |
|[5A 58 5A 5A 54

None

In [152]:
raw_stream.stop()

### Convert Key Value pairs to strings

In [153]:
string_stream_df = stream_df \
    .withColumn("key", stream_df["key"].cast(StringType())) \
    .withColumn("value", stream_df["value"].cast(StringType()))

In [154]:
string_stream = string_stream_df \
    .writeStream \
    .format("memory") \
    .queryName("string_stocktrades_view") \
    .start()

In [156]:
clear_output(wait=True)
display(spark.sql('SELECT key, value FROM string_stocktrades_view').show(20, False))
time.sleep(1)

+-----+----------------------------------+
|key  |value                             |
+-----+----------------------------------+
|ZVZZT|    BUY�
ZVZZT�ABC123User_1 |
|ZVZZT|    BUY�8
ZVZZT�LMN456User_8 |
|ZWZZT|    SELL�)
ZWZZT�ABC123User_2|
|ZJZZT|    BUY�)
ZJZZT�XYZ789User_8 |
|ZVZZT|    BUY�C
ZVZZT�XYZ789User_4 |
|ZXZZT|    SELL�
ZXZZT�ABC123User_9|
|ZVZZT|    SELL�#
ZVZZT�LMN456User_5|
|ZBZX |    SELL�ZBZX�LMN456User_5 |
|ZVZZT|    BUY�
ZVZZT(ABC123User_3  |
|ZTEST|    SELL�J
ZTEST�	XYZ789User_6|
|ZBZX |    BUY�ZBZXABC123User_8  |
|ZJZZT|    SELL�(
ZJZZT�LMN456User_7|
|ZJZZT|    SELL�C
ZJZZT�XYZ789User_3|
|ZTEST|    SELL�
ZTEST�LMN456User_6|
|ZVV  |    SELL�?ZVV�LMN456User_9  |
|ZBZX |    SELL�ZBZX�LMN456User_2 |
|ZTEST|    BUY�0
ZTEST�
LMN456User_3 |
|ZWZZT|    SELL�(
ZWZZT�ABC123User_4|
|ZXZZT|    BUY�&
ZXZZT�XYZ789User_8 |
|ZXZZT|    BUYR
ZXZZT�ABC123User_1  |
+-----+----------------------------------

None

In [157]:
string_stream.stop()

## Transformation

In [73]:
schema_stocktrades = StructType([
        StructField("side", StringType(),  True),
        StructField("quantity", IntegerType(),  True),
        StructField("price", IntegerType(),  True),
        StructField("symbol", StringType(),  True),
        StructField("account", StringType(), True),
         StructField("", StringType(), True)
    
])

In [74]:
json_stream_df = string_stream_df\
    .withColumn("value", F.from_json("value", schema_stocktrades))

In [75]:
json_stream_df.printSchema()

root
 |-- key: string (nullable = true)
 |-- value: struct (nullable = true)
 |    |-- side: string (nullable = true)
 |    |-- quantity: integer (nullable = true)
 |    |-- price: integer (nullable = true)
 |    |-- symbol: string (nullable = true)
 |    |-- account: string (nullable = true)
 |    |-- userid: string (nullable = true)
 |-- topic: string (nullable = true)
 |-- partition: integer (nullable = true)
 |-- offset: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- timestampType: integer (nullable = true)



In [68]:
json_stream = json_stream_df \
    .writeStream \
    .format("memory") \
    .queryName("extract_stocktrades_view") \
    .start()

In [71]:
clear_output(wait=True)
display(spark.sql('SELECT * FROM extract_stocktrades_view').show(20))
time.sleep(1)

+-----+--------------------+-----------+---------+------+--------------------+-------------+
|  key|               value|      topic|partition|offset|           timestamp|timestampType|
+-----+--------------------+-----------+---------+------+--------------------+-------------+
|ZVZZT|{null, null, null...|stocktrades|        0|     0|2021-06-05 05:39:...|            0|
|ZVZZT|{null, null, null...|stocktrades|        0|     1|2021-06-05 05:39:...|            0|
|ZWZZT|{null, null, null...|stocktrades|        0|     2|2021-06-05 05:39:...|            0|
|ZJZZT|{null, null, null...|stocktrades|        0|     3|2021-06-05 05:39:...|            0|
|ZVZZT|{null, null, null...|stocktrades|        0|     4|2021-06-05 05:39:...|            0|
|ZXZZT|{null, null, null...|stocktrades|        0|     5|2021-06-05 05:39:...|            0|
|ZVZZT|{null, null, null...|stocktrades|        0|     6|2021-06-05 05:39:...|            0|
| ZBZX|{null, null, null...|stocktrades|        0|     7|2021-06-05 05

None

In [72]:
json_stream.stop()

In [142]:
spark.stop()